# Webscraping ZAMG Daten

## Einleitung

Um für unser Projekt aktuelle Wetterdaten zu erhalten, haben wir von der Seite Zamg... etc. via Webscraping unsere Daten gewonnen:
 da wir uns für unsere Endauswertung für 4 Wetterstationen in Niederösterreich und Burgenland interessieren, haben wir sowohl die Seite von Niederösterreich als auch die Seite von Burgenland verwendet.
 
 
weiterer Text zu den Daten....

## Datenmanagement

Zunächst werden die notwenigen Librariers geladen.

In [8]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import csv
#import psycopg2
import pandas as pd
import datetime as dt

Nun wird die Url erzeugt - hier am Beispiel für Niederösterreich (analog wurde mit Burgenland verfahren) - und Beautiful Soap übergeben.

In [ ]:
html = urlopen('https://www.zamg.ac.at/cms/de/wetter/wetterwerte-analysen/niederoesterreich')
#html = urlopen('https://www.zamg.ac.at/cms/de/wetter/wetterwerte-analysen/burgenland')
bs = BeautifulSoup(html, 'html.parser')
#print(bs)

Anschließend wir durch die Seite navigiert, bis wir die richtige Tabelle finden.

In [10]:
table = bs.find('table', attrs={'class':'dynPageTable'})
table_body = table.find('tbody')
#print(table)

In [11]:
data = []
rows = bs.find_all("tr", {"class": {"dynPageTableLine2", "dynPageTableLine1"}})
#print(mydivs)
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols] # get rid of html elements
    data.append([ele for ele in cols])

Das Ergebnis wird in einem Panda-Dataframe gespeichert spreichern und nach csv konvertiert.

In [12]:
df = pd.DataFrame(data)
df.columns = ['location', 'height', 'temp', 'hum', 'wind', 'wind_max', 'rain', 'sun', 'press' ]
df['timestamp'] = pd.Series([dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S")] * len(df))

In [13]:
df.to_csv('zamg.csv', sep=";", encoding="utf-8", index=False, mode='a', header=False)

Dieser Code lief auf einem Linuxsystem einmal pro Stunde über einen Zeitraum von 21.5.2022 bis 24.06.2022 für Niederösterreich sowie vom 25.5.2022 bis 24.6.2022 für Burgenland, um die aktuellen Wetterdaten aus dem Burgenland und Niederöstereich zu aggregieren.